In [ ]:
json_request={
    "parameter": {
        "data": {
            "idConfRoute": "2b3aac5f-f786-4430-b37f-f75ccd247960",
            "forbiddenPort": "80a4b48e-2bbb-42f8-b0f5-aed166f81eb3, bb5c997e-f3ac-4b41-92d1-23458aa5c1f1",
            "status": "RUNNING",
            "runningTime": 0,
            "accuracy": 0,
            "method": "string",
            "trial": 0,
            "routeDetail": [
            {
                "idConfRouteDetail": "ff0dba1c-6211-4b52-9cea-c7a49822e05a",
                "idShip": "f3b7a6b0-21a3-45e5-82dc-0eaaec39ede4",  
                "lastLocationPort": "05aa9d6c-6999-4f7a-b4d6-7dddca9db337",
                "originPort": "19887a24-9011-425f-bc0c-5f5cd512ba5a",
                "destinationPort": "",
                "mustVisitPort": "",
                "minPort": 2,
                "maxPort": 10,
                "minComDays": 1,
                "maxComDays": 14,
                "regionRule": "NOT_SPECIFIED",
                "roundTrip": 1
            },
            {
                "idConfRouteDetail": "d1339ee1-9472-4d34-a722-000b9edc4779",
                "idShip": "d4787f56-56d0-45ae-8822-ce55b720357a",  
                "lastLocationPort": "19887a24-9011-425f-bc0c-5f5cd512ba5a",
                "originPort": "",
                "destinationPort": "3c42cdbb-1dd7-4214-99d3-ffabc2c2a93d",
                "mustVisitPort": "6a01e7c5-05e6-4ab7-a706-f0936f32a837, 3c42cdbb-1dd7-4214-99d3-ffabc2c2a93d",
                "minPort": 3,
                "maxPort": 50,
                "minComDays": 1,
                "maxComDays": 28,
                "regionRule": "NOT_SPECIFIED",
                "roundTrip": 1
            }]
        }
    }
}

In [ ]:
import pandas as pd
import time
from typing import List,Callable,Tuple
from collections import namedtuple
from functools import partial
import import_ipynb
from data_preparation import *
from general_function import *
# from calculator import *

input_request = input_df(json_request)

idConf = input_request[0]
forbidden = input_request[1]
df_input = input_request[2]

##EXECUTE GLOBAL VARIABLES

#master port, exclude forbiddenport
df_port_all = retrieve_port('001')
df_port = df_port_all[~df_port_all['id'].isin([forbidden])]

#master ship, include input ship only
df_ship_all = retrieve_ship('001')
df_ship = df_ship_all[df_ship_all['id'].isin(df_input.idShip)]
df_ship=df_ship.fillna(0)
ship_input=pd.merge(df_ship,df_input, how='inner', left_on=['id'], right_on=['idShip'])

#master port distance
df_portdistance = retrieve_portdistance()

#revenue forecast
df_revenue = retrieve_revenue(2)

#config price cargo
df_pricecargoconfig = retrieve_pricecargoconfig()

#config low peak
df_lowpeak = retrieve_lowpeak()

#config basefare
df_basefare = retrieve_basefare()

#config cost rule
df_rulecost = retrieve_rulecost()

#price cargo flat
df_cargoflat = retrieve_cargoflat()

#price adjustment
df_adjustment = retrieve_adjustment()

#config price pnp
df_priceconfig = retrieve_priceconfig()

# %store df_port
# %store df_ship
# %store df_portdistance
# %store df_revenue
# %store df_pricecargoconfig
# %store df_lowpeak
# %store df_basefare
# %store df_rulecost
# %store df_cargoflat
# %store df_adjustment
# %store df_priceconfig

In [ ]:
start = time.time()
output_low = generate_route_alpha_ver(ship_input, df_port, df_ship, df_portdistance, df_adjustment,
                                      df_basefare, df_cargoflat, df_lowpeak, df_pricecargoconfig, 
                                      df_priceconfig, df_revenue, df_rulecost)
header_low = output_low[0]
est_low = output_low[1]
cost_low = output_low[2]
end = time.time()        
print(f"time spent for regular optimiser: {end-start}s")

In [ ]:
start = time.time()
output_peak = generate_route_alpha_ver(ship_input, df_port, df_ship, df_portdistance, df_adjustment,
                                      df_basefare, df_cargoflat, df_lowpeak, df_pricecargoconfig, 
                                      df_priceconfig, df_revenue, df_rulecost, season='PEAK')
header_peak = output_peak[0]
est_peak = output_peak[1]
cost_peak = output_peak[2]
end = time.time()
print(f"time spent for peak optimiser: {end-start}s")

header_all = pd.concat([header_low,header_peak], axis=0)
est_all = pd.concat([est_low,est_peak], axis=0)
cost_all = pd.concat([cost_low,cost_peak], axis=0)
